# Importações

In [ ]:
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import unidecode
import Database
import json


In [ ]:
#Re-importanto a biblioteca porque foi modificada - pode ser ignorado
#import importlib
#importlib.reload(Database)

In [ ]:
db = Database.Database('CNPq')

## Carregando tabela de indicadores

In [ ]:
engine = Carga.db_engine()
dt = pd.read_sql("indicadores_pareamento", engine)
dt.size

## Pegando lista de Ids já realizados

In [ ]:
ids_já_feitos = []

In [ ]:
#sql = 'SELECT DISTINCT id FROM pareamentos;'
#ids_já_feitos = db.query(sql)

In [ ]:
#ids_já_feitos = [num[0] for num in ids_já_feitos]

In [ ]:
#len (ids_já_feitos)

In [ ]:
db.commit()

## Retirada dos não doutores - Existem 3 não doutores contemplados!!!

In [ ]:
dt = dt[pd.notnull(dt.ano_doutorado)]

## Normalizando as Tabelas

In [ ]:
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)

In [ ]:
dt.area_demanda_bruta = dt.area_demanda_bruta.fillna('nenhum')
dt.areas_conhecimento = dt.areas_conhecimento.fillna('nenhum')
dt.tipos_vinculo = dt.tipos_vinculo.fillna('nenhum')
dt.enquadramento_vinculo = dt.enquadramento_vinculo.fillna('nenhum')

In [ ]:
dt.area_demanda_bruta = dt.area_demanda_bruta.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()

In [ ]:
dt.areas_conhecimento = dt.areas_conhecimento.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()

In [ ]:
dt.tipos_vinculo = dt.tipos_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()

In [ ]:
dt.enquadramento_vinculo = dt.enquadramento_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()

## Criando tabelas parciais

In [ ]:
financiados = dt[~pd.isnull(dt.pgtos)]

In [ ]:
len(financiados.id.unique())

In [ ]:
dt = dt[pd.isnull(dt.pgtos)]

In [ ]:
erros = dt[dt.id.isin(financiados.id.unique())]

In [ ]:
dt = dt[~dt.id.isin(financiados.id.unique())]

## Fazendo Listas de Distribuição

area = []
for id in financiados.id.unique():
    financiado = financiados[financiados.id == id]

    tipos_vinculo = financiado.tipos_vinculo.unique()
    
    if not tipos_vinculo == None and len(tipos_vinculo) > 0:   
        if tipos_vinculo[0].find('servidor_publico') > -1:
            enquadramento = 'servidor_publico'
        elif tipos_vinculo[0].find('celetista') > -1:
            enquadramento = 'celetista'
        elif tipos_vinculo[0].find('professor_visitante') > -1:
            enquadramento = 'professor_visitante' 
        elif tipos_vinculo[0].find('colaborador') > -1:
            enquadramento = 'colaborador' 
        elif tipos_vinculo[0].find('bolsista') > -1:
            enquadramento = 'bolsista' 
        elif tipos_vinculo[0].find('livre') > -1:
            enquadramento = 'livre' 
        elif tipos_vinculo[0].find('outro') > -1:
            enquadramento = 'outro'     
    else: enquadramento = 'nenhum'
    
    area.append({
        'id': id,
        'enquadramento_vinculo': tipos_vinculo[0],
        'tipo_vinculo': enquadramento
    })
print (area)

financiados

tipos_vinculos = pd.DataFrame(area)

tipos_vinculos.tipo_vinculo.explode().value_counts()

2194 + 590 + 206 + 885


## Lista de Distribuição Tipo Vínculo

area = []
for id in financiados.id.unique():
    financiado = financiados[financiados.id == id]

    enquadramento_vinculo = financiado.enquadramento_vinculo.unique()
    
    if not enquadramento_vinculo == None and len(enquadramento_vinculo) > 0:
        if enquadramento_vinculo[0].find('professor') > -1:
            vinculo = 'professor'
        elif enquadramento_vinculo[0].find('outro') > -1:
            vinculo = 'outro'    
        elif enquadramento_vinculo[0].find('livre') > -1:
            vinculo = 'livre'      
    else: vinculo  = 'nenhum'
    
    area.append({
        'id': id,
        'enquadramento_vinculo': tipos_vinculo[0],
        'tipo_vinculo': vinculo
    })
print (area)

tipos_vinculos = pd.DataFrame(area)

tipos_vinculos.tipo_vinculo.explode().value_counts()

# Pareando

min_para_parear = 3
num_pareamentos = 3
erros_ids_inexistentes_pareamento = []
lista_pareamentos = []
for id in financiados.id.unique():
    #Pegando dados do ID
    print('Fazendo id: ', id)
    financiado = financiados[financiados.id == id]
    if financiado.empty:
        print('ERRO. Id não encontrado na tabela financiados: ', id)
        erros_ids_inexistentes_pareamento.append(id)
        with open('d:\erros_ids_inexistentes_pareamento.json', 'w') as f:
            json.dump(erros_ids_inexistentes_pareamento, f)
        continue      
    
    #Pegando áreas do Conhecimento
    financiado.areas_conhecimento.iloc[0].split(',')
    
    #Pegando Regiões
    região = financiado.iloc[0].uf
    tipo_região = None
    if not região == None:
        if região in ('SP'):
            tipo_região = 1
        elif região in ('MG', 'RS', 'RJ', 'PR'):
            tipo_região = 2
        else:
            tipo_região = 3
    
    #Pegando ano do Indicador a ser usado
    if financiado.chamada.str.contains('2012', na=False).unique()[0] == True : ano = 2012
    elif financiado.chamada.str.contains('2013', na=False).unique()[0] == True : ano = 2013
    elif financiado.chamada.str.contains('2014', na=False).unique()[0] == True : ano = 2014
    else: ano = None

    #Pegando Faixa do Financiado
    if financiado.chamada.str.contains('Faixa A', na=False).unique()[0] == True : faixa = 'A'
    elif financiado.chamada.str.contains('Faixa B', na=False).unique()[0] == True : faixa = 'B'
    elif financiado.chamada.str.contains('Faixa c', na=False).unique()[0] == True : faixa = 'C'
    else: faixa = None
        
    #Pegando o vínculo do Financiado
    tipos_vinculo = financiado.tipos_vinculo.unique()
    if not tipos_vinculo == None and len(tipos_vinculo) > 0:   
        if tipos_vinculo[0].find('servidor_publico') > -1:
            é_servidor = 'servidor_publico'
        else:
            é_servidor = 'outros'   
    else: é_servidor = 'nenhum'
        
    #Fazendo o Pareamento
    tipos_de_pareamento = []  #Variável que vai indicar quais os tipos de pareamento realizados
    
    #1. Pareando pela área de conhecimento
    pareados2 = dt.loc[(dt.areas_conhecimento.str.contains(financiado.area_demanda_bruta.unique()[0].split('(')[0].strip(), na=False))]
    tipos_de_pareamento.append('area_demanda_bruta')
                       
    #2. Pareando pelo Vínculo
    if pareados2.size > min_para_parear:
        pareados = pareados2
        if é_servidor == 'servidor_publico':
            pareados2 = pareados.loc[(dt.tipos_vinculo.str.contains('servidor_publico', na=False))]
        else:
            pareados2 = pareados.loc[~(dt.tipos_vinculo.str.contains('servidor_publico', na=False))]
        tipos_de_pareamento.append('tipos_vinculo')
                       
    
    #3. Pareando por ano de doutorado -> Faixa de 20 anos
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[
            (dt.ano_doutorado < financiado.ano_doutorado.unique()[0] + 10) &
            (dt.ano_doutorado > financiado.ano_doutorado.unique()[0] - 10)
        ]
        tipos_de_pareamento.append('ano_doutorado_10')
                       
    #3. Pareando por ano de doutorado -> Faixa de 10 anos
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[
            (dt.ano_doutorado < financiado.ano_doutorado.unique()[0] + 5) &
            (dt.ano_doutorado > financiado.ano_doutorado.unique()[0] - 5)
        ]
        tipos_de_pareamento.append('ano_doutorado_05')
                       
    #4. Pareando por sexo
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[(dt.sexo == financiado.sexo.unique()[0])]
        tipos_de_pareamento.append('sexo')
                       
    #5. Pareando por região
    if pareados2.size > min_para_parear:
        pareados = pareados2
        if tipo_região == 1:
            pareados2 = pareados.loc[pareados.uf.isin(('SP',))]
        elif tipo_região == 2:
            pareados2 = pareados.loc[pareados.uf.isin(('MG', 'RS', 'RJ', 'PR'))]
        elif tipo_região == 3:
            pareados2 = pareados.loc[~pareados.uf.isin(('SP', 'MG', 'RS', 'RJ', 'PR'))]

        tipos_de_pareamento.append('uf')
                       
    #6. Pareando por erros quadráticos
    if pareados2.size > min_para_parear:
        pareados = pareados2
        tipos_de_pareamento.append('erro')            
                                             
        #Fezendo lista de erros quadráticos
        ids_pareados = []
        if not pareados.empty:

            #Calculando os erros dos grupos
            # "grupo" é um grupo de indicadores.
            for grupo in financiado.grupo.unique():

                #Pegando o valor do Financiado
                ind_financiado = financiado.loc[(financiado.grupo == grupo)]['qty_' + str(ano)].iloc[0]

                #Verificando o erro dos pareados restantes
                for id_pareado in pareados.id.unique():
                    ind_pareado = pareados.loc[
                        (pareados.grupo == grupo)
                        & (pareados.id == id_pareado)
                        ]['qty_' + str(ano)]
                    if len(ind_pareado) > 0:
                        ind_pareado = ind_pareado.iloc[0]
                    else:
                        ind_pareado = 0
                    erro = {
                        'id': id,
                        'id_pareado': id_pareado,
                        'grupo': grupo,
                        'valor': ind_pareado,
                        'erro2': (ind_financiado - ind_pareado)**2
                    }
                    ids_pareados.append(erro)

            #Acrescentando Anos de Doutor como um Indicador
            for id_pareado in pareados.id.unique():
                ind_pareado = pareados.ano_doutorado.loc[
                    (pareados.id == id_pareado)
                    ].unique()
                #print(ind_pareado, type(ind_pareado))
                if len(ind_pareado) > 0:
                    ind_pareado = ind_pareado[0]
                else:
                    ind_pareado = 0
                erro = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'grupo': "Anos de Doutor",
                    'valor': ind_pareado,
                    'erro2': (financiado.ano_doutorado.iloc[0] - ind_pareado)**2
                }
                ids_pareados.append(erro)

            #Acrescentando Áreas do Conhecimento em Comum como outro indicador
            areas_conhecimento_financiado = financiado.areas_conhecimento.iloc[0]
            if not areas_conhecimento_financiado == None:
                areas_conhecimento_financiado = areas_conhecimento_financiado.split(',')
                total_areas = len(areas_conhecimento_financiado)

                for id_pareado in pareados.id.unique():
                    areas_em_comum = 0
                    lista_areas_conhecimento = pareados.loc[pareados.id == id_pareado].areas_conhecimento
                    if len(lista_areas_conhecimento) > 0 and not lista_areas_conhecimento.iloc[0] == None:
                        lista_areas_conhecimento = lista_areas_conhecimento.iloc[0].split(',')
                        for area in areas_conhecimento_financiado:
                            if area in lista_areas_conhecimento:
                                areas_em_comum += 1
                    erro = {
                        'id': id,
                        'id_pareado': id_pareado,
                        'grupo': 'areas_conhecimento',
                        'valor': areas_em_comum,
                        'erro2': (total_areas - areas_em_comum)**2
                    }
                    ids_pareados.append(erro)    

        #Calculando o Erro
        erro = pd.DataFrame(ids_pareados)
        if not erro.empty:
            desvio_padrão = []
            for id_pareado in erro.id_pareado.unique():
                erro_id = erro.loc[erro.id_pareado == id_pareado]
                if not erro_id.empty:
                    soma_erro = erro_id.erro2.sum()
                    num_erros = len(erro_id.index)
                    err = {
                        'id_pareado': id_pareado,
                        'erro': np.sqrt(soma_erro/num_erros),
                    }
                    
                    desvio_padrão.append(err)
                    lista_pareamentos.append(err)

                    #db.insert_dict("pareamentos", err, on_conflict=['id', 'id_pareado'])
            pareados_por_desvio_padrão = pd.DataFrame(desvio_padrão)
            if not pareados_por_desvio_padrão.empty:
                lista_pareados = pareados_por_desvio_padrão.sort_values(by=['erro']).iloc[:3]
                #print(lista_pareados)
    else:
        desvio_padrão = []
        x = 0
        for id_pareado in pareados.id.unique():
            err = {
                'id_pareado': id_pareado,
                'erro': 'Nan',
            }
            desvio_padrão.append(err)
            x += 1
            if x > 3: break
        pareados_por_desvio_padrão = pd.DataFrame(desvio_padrão)
        if not pareados_por_desvio_padrão.empty:
            lista_pareados = pareados_por_desvio_padrão.sort_values(by=['erro']).iloc[:3]
            #print(lista_pareados)
                      
    lista_pareado_para_incluir_no_bd = {
        'id': id,
        'tipos_pareamento': tipos_de_pareamento
    }
    x = 1
    for pareado in desvio_padrão:
        lista_pareado_para_incluir_no_bd['pareado_' + str(x)] = pareado['id_pareado']
        lista_pareado_para_incluir_no_bd['erro_' + str(x)] = pareado['erro']
        x += 1
        if x > 3: break
    print(lista_pareado_para_incluir_no_bd)
        
                


In [ ]:
#
#
#
## Fazendo Pareamento
#
#
#

min_para_parear = 3 # -> número de pessoas pareadas de modo fixo, sem usar os erros
num_pareamentos = 3# -> número de pessoas pareadas ao final -> vai usar erros quadráticos para chegar nesse número

lista_pareamentos = [] # -> Lista com todos os pareados -> Útil apenas para conferência depois

for id in financiados.id.unique():

    #Pegando dados do ID
    print('Fazendo id: ', id)
    financiado = financiados[financiados.id == id]
    
    #Pegando áreas do Conhecimento do Financiado
    financiado.areas_conhecimento.iloc[0].split(',')
    
    #Pegando Regiões do Financiado
    região = financiado.iloc[0].uf
    tipo_região = None
    if not região == None:
        if região in ('SP'):
            tipo_região = 1
        elif região in ('MG', 'RS', 'RJ', 'PR'):
            tipo_região = 2
        else:
            tipo_região = 3
    
    #Pegando ano do Indicador a ser usado do Financiado -> Útil para calcular erro -> Vai ser usado ao calcular o impacto
    if financiado.chamada.str.contains('2012', na=False).unique()[0] == True : ano = 2012
    elif financiado.chamada.str.contains('2013', na=False).unique()[0] == True : ano = 2013
    elif financiado.chamada.str.contains('2014', na=False).unique()[0] == True : ano = 2014
    else: ano = None

    #Pegando Faixa do Financiado do Financiado -> Inútil aqui -> Vai ser usado ao calcular o impacto
    if financiado.chamada.str.contains('Faixa A', na=False).unique()[0] == True : faixa = 'A'
    elif financiado.chamada.str.contains('Faixa B', na=False).unique()[0] == True : faixa = 'B'
    elif financiado.chamada.str.contains('Faixa c', na=False).unique()[0] == True : faixa = 'C'
    else: faixa = None
        
    #Pegando o vínculo do Financiado
    tipos_vinculo = financiado.tipos_vinculo.unique()
    if not tipos_vinculo == None and len(tipos_vinculo) > 0:   
        if tipos_vinculo[0].find('servidor_publico') > -1:
            é_servidor = 'servidor_publico'
        else:
            é_servidor = 'outros'   
    else: é_servidor = 'nenhum'
    
    #   
    #Fazendo o Pareamento
    #
    tipos_de_pareamento = []  #Variável que vai indicar quais os tipos de pareamento realizados
    
    #1. Pareando pela área de conhecimento
    pareados2 = dt.loc[(dt.areas_conhecimento.str.contains(financiado.area_demanda_bruta.unique()[0].split('(')[0].strip(), na=False))]
    tipos_de_pareamento.append('area_demanda_bruta')
                       
    #2. Pareando pelo Vínculo
    if pareados2.size > min_para_parear:
        pareados = pareados2
        if é_servidor == 'servidor_publico':
            pareados2 = pareados.loc[(dt.tipos_vinculo.str.contains('servidor_publico', na=False))]
        else:
            pareados2 = pareados.loc[~(dt.tipos_vinculo.str.contains('servidor_publico', na=False))]
        tipos_de_pareamento.append('tipos_vinculo')
                       
    
    #3. Pareando por ano de doutorado -> Faixa de 20 anos
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[
            (dt.ano_doutorado < financiado.ano_doutorado.unique()[0] + 10) &
            (dt.ano_doutorado > financiado.ano_doutorado.unique()[0] - 10)
        ]
        tipos_de_pareamento.append('ano_doutorado_10')
                       
    #3. Pareando por ano de doutorado -> Faixa de 10 anos
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[
            (dt.ano_doutorado < financiado.ano_doutorado.unique()[0] + 5) &
            (dt.ano_doutorado > financiado.ano_doutorado.unique()[0] - 5)
        ]
        tipos_de_pareamento.append('ano_doutorado_05')
                       
    #4. Pareando por sexo
    if pareados2.size > min_para_parear:
        pareados = pareados2
        pareados2 = pareados.loc[(dt.sexo == financiado.sexo.unique()[0])]
        tipos_de_pareamento.append('sexo')
                       
    #5. Pareando por região
    if pareados2.size > min_para_parear:
        pareados = pareados2
        if tipo_região == 1:
            pareados2 = pareados.loc[pareados.uf.isin(('SP',))]
        elif tipo_região == 2:
            pareados2 = pareados.loc[pareados.uf.isin(('MG', 'RS', 'RJ', 'PR'))]
        elif tipo_região == 3:
            pareados2 = pareados.loc[~pareados.uf.isin(('SP', 'MG', 'RS', 'RJ', 'PR'))]

        tipos_de_pareamento.append('uf')
                       
    #6. Pareando por erros quadráticos
    pareados = pareados2
    tipos_de_pareamento.append('erro')            
                                            
    #Fezendo lista de erros quadráticos
    ids_pareados = []
    if not pareados.empty:

        #Calculando os erros dos grupos
        # "grupo" é um dos grupos de indicadores.
        for grupo in financiado.grupo.unique():

            #Pegando o valor do Financiado
            ind_financiado = financiado.loc[(financiado.grupo == grupo)]['qty_' + str(ano)].iloc[0]

            #Verificando o erro dos pareados restantes
            for id_pareado in pareados.id.unique():
                ind_pareado = pareados.loc[
                    (pareados.grupo == grupo)
                    & (pareados.id == id_pareado)
                    ]['qty_' + str(ano)]
                if len(ind_pareado) > 0:
                    ind_pareado = ind_pareado.iloc[0]
                else:
                    ind_pareado = 0
                erro = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'grupo': grupo,
                    'valor': ind_pareado,
                    'erro2': (ind_financiado - ind_pareado)**2
                }
                ids_pareados.append(erro)

        #Acrescentando Anos de Doutor como um Indicador
        for id_pareado in pareados.id.unique():
            ind_pareado = pareados.ano_doutorado.loc[
                (pareados.id == id_pareado)
                ].unique()
            #print(ind_pareado, type(ind_pareado))
            if len(ind_pareado) > 0:
                ind_pareado = ind_pareado[0]
            else:
                ind_pareado = 0
            erro = {
                'id': id,
                'id_pareado': id_pareado,
                'grupo': "Anos de Doutor",
                'valor': ind_pareado,
                'erro2': (financiado.ano_doutorado.iloc[0] - ind_pareado)**2
            }
            ids_pareados.append(erro)

        #Acrescentando Áreas do Conhecimento em Comum como outro indicador
        areas_conhecimento_financiado = financiado.areas_conhecimento.iloc[0]
        if not areas_conhecimento_financiado == None:
            areas_conhecimento_financiado = areas_conhecimento_financiado.split(',')
            total_areas = len(areas_conhecimento_financiado)

            for id_pareado in pareados.id.unique():
                areas_em_comum = 0
                lista_areas_conhecimento = pareados.loc[pareados.id == id_pareado].areas_conhecimento
                if len(lista_areas_conhecimento) > 0 and not lista_areas_conhecimento.iloc[0] == None:
                    lista_areas_conhecimento = lista_areas_conhecimento.iloc[0].split(',')
                    for area in areas_conhecimento_financiado:
                        if area in lista_areas_conhecimento:
                            areas_em_comum += 1
                erro = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'grupo': 'areas_conhecimento',
                    'valor': areas_em_comum,
                    'erro2': (total_areas - areas_em_comum)**2
                }
                ids_pareados.append(erro)    

    #Calculando o Desvio Padrão por id pareado
    erro = pd.DataFrame(ids_pareados)
    if not erro.empty:
        desvio_padrão = []  # -> VARIÁVEL QUE VAI CONTER OS IDS PAREADOS
        for id_pareado in erro.id_pareado.unique():
            erro_id = erro.loc[erro.id_pareado == id_pareado]
            if not erro_id.empty:
                soma_erro = erro_id.erro2.sum()
                num_erros = len(erro_id.index)
                err = {
                    'id_pareado': id_pareado,
                    'erro': np.sqrt(soma_erro/num_erros),
                }
                
                desvio_padrão.append(err)
                lista_pareamentos.append(err)

                #db.insert_dict("pareamentos", err, on_conflict=['id', 'id_pareado'])
        pareados_por_desvio_padrão = pd.DataFrame(desvio_padrão)
        if not pareados_por_desvio_padrão.empty:
            lista_pareados = pareados_por_desvio_padrão.sort_values(by=['erro']).iloc[:3]
            #print(lista_pareados)
                      
    lista_pareado_para_incluir_no_bd = {
        'id': id,
        'ano': ano,
        'faixa': faixa,
        'tipos_pareamento': tipos_de_pareamento
    }
    x = 1
    for pareado in desvio_padrão:
        lista_pareado_para_incluir_no_bd['pareado_' + str(x)] = pareado['id_pareado']
        lista_pareado_para_incluir_no_bd['erro_' + str(x)] = pareado['erro']
        x += 1
        if x > 3: break
    print(lista_pareado_para_incluir_no_bd)
        

In [ ]:
db.commit()

### Erro para parar o código

In [ ]:
efghrertherthhter
print estou na fazendo sem parênteses ou aspas

In [ ]:
pareados

In [ ]:
pareados = dt.loc[(dt.areas_conhecimento.str.contains(financiado.area_demanda_bruta.unique()[0].split('(')[0].strip(), na=False))]

In [ ]:
pareados

In [ ]:
financiado.area_demanda_bruta.unique()[0].split('(')[0]